# 3 LAYER NET.

BROKEN AI
3 layer per allenamento con MNIST

In [ ]:
import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist


# split dataset into training and test set
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train1=x_train
y_train1=y_train
x_test1=x_test
y_test1=y_test

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

In [ ]:
x_train = x_train.reshape(60000, 784)

x_test = x_test.reshape(10000, 784)

In [ ]:
from keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes=10)

y_test = to_categorical(y_test, num_classes=10)

In [ ]:
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

In [ ]:
model = Sequential()
model.add(Dense(10, activation='sigmoid', input_shape=(784,)))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='softmax'))


In [ ]:
model.summary()

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer="sgd",#oppure adam per avere learning rate adattivo
              metrics = ['accuracy'])

In [ ]:
import time
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping 
keras_callbacks = ReduceLROnPlateau(
    monitor= 'val_loss',     # Metrica da monitorare (qui è la loss di validazione) 
    factor=0.95,             # Riduci il learning rate: Moltiplicatore per ridurre il learning rate (nuovo_lr = lr * factor)
    patience=1,             # Aspetta 1 epoche senza miglioramenti prima di ridurre LR
    verbose=1,              # Mostra un messaggio quando il learning rate viene ridotto
    min_lr=0,             # Learning rate minimo
    min_delta=0.05      # min_delta=quantità minima di miglioramento richiesta per considerare il valore della perdita come un miglioramento
)


#per modificare epochs e batch_size modificare rispettivamente 'e' e 'b' (verranno modificati anche nella cnn)
e=30
b=700



#altro callbacks con Earlystopping
keras_callbacks1 = [EarlyStopping(monitor='val_loss', patience=6, verbose=1, min_delta=0.001)]
#min_delta=quantità minima di miglioramento richiesta per considerare il valore della perdita come un miglioramento


start_time = time.time() #registra tempo di inizio
history = model.fit(x_train, y_train, epochs=e, batch_size=b, verbose=1,
                    validation_split=0.2, callbacks=[keras_callbacks,keras_callbacks1])

end_time = time.time() ##registra tempo di fine


#model.fit(x_train, y_train, batch_size=60000, epochs=100,validation_split=0.2)


In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', round(test_acc,4))
print('Test loss:', round(test_loss,4))

In [ ]:
import matplotlib.pyplot as plt
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=30)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Actual class')
    plt.xlabel('Predicted class')
   
    plt.show()

In [ ]:
from collections import Counter
from sklearn.metrics import confusion_matrix
import itertools

# Predict the values from the validation dataset
Y_pred = model.predict(x_test)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred, axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_test, axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(10))



In [ ]:
#plot loss-rate
val_losses=history.history["val_loss"]
train_losses=history.history["loss"]

epochs = range(1, len(val_losses) + 1)

plt.figure()
plt.title("Training loss")
plt.plot(epochs,val_losses,c="red",label="Validation")
plt.plot(epochs,train_losses,c="orange",label="Training")
plt.xlabel("Epochs")
plt.ylabel("Cross entropy")
plt.legend()
plt.savefig("loss_ANN.png")

# rete modello (CNN)
implemento una rete modello perfettamente funzionante (tale cnn con 8 layer è più stabile e meglio ottimizzata, ottima accuracy e loss basso).
tale modello può essere utile per un eventuale confronto dei dati in output della rete ANN con un set di dati che simuli 'le risposte di un paziente perfettamente sano'
NOTA:per quanto ottima la CNN non è perfetta potrà sbagliare qualche test, quindi questo introduce un certo margine di errore per la rete ANN. 

In [ ]:
from keras.layers import Dense,Conv2D,Flatten,Dropout,MaxPooling2D

x_train_cnn=x_train1.reshape(x_train1.shape[0],x_train1.shape[1],x_train1.shape[2],1)
x_test_cnn=x_test1.reshape(x_test1.shape[0],x_test1.shape[1],x_test1.shape[2],1)

y_train_cat=to_categorical(y_train1)

    # definiamo il modello in maniera funzionale
cnn= Sequential()
cnn.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(28, 28, 1)))
cnn.add(MaxPooling2D(strides=2))
cnn.add(Conv2D(filters=48, kernel_size=(5,5), padding='valid', activation='relu'))
cnn.add(MaxPooling2D(strides=2))
cnn.add(Flatten())
cnn.add(Dense(256, activation='relu'))
cnn.add(Dense(84, activation='relu'))
cnn.add(Dense(10, activation='softmax'))

    # definizione dell'ottimizzatore
    #opt=Adam(learning_rate=1e-3) #algoritmo di ottimizzazione, learning rate indica il tasso di apprendimento

    # compiliamo il modello prima dell'addestramento
cnn.compile(optimizer="adam",
                       loss='categorical_crossentropy', #loss function
                       metrics=['accuracy'])
keras_callbacks = [
EarlyStopping(monitor='val_loss', patience=5, verbose=1, min_delta=0.001)#quantità minima di miglioramento richiesta per considerare il valore della perdita come un miglioramento
]
start_time = time.time() #registra tempo di inizio
history = cnn.fit(x_train_cnn, y_train_cat, epochs=e, batch_size=b, verbose=1,
                    validation_split=0.2, callbacks=keras_callbacks)
    
   

end_time = time.time() ##registra tempo di fine
    
cnn_predictions = cnn.predict(x_test1)
y_test_cnn = to_categorical(y_test1, num_classes=10)

test_loss_R, test_acc_R = cnn.evaluate(x_test_cnn, y_test_cnn)
print('Test accuracy cnn:', round(test_acc_R,4))
print('Test loss cnn:', round(test_loss_R,4))

# simulazione 
comportamento di un paziente (non bravo) a cui vengono proposte immagini da classificare (MNIST) e confronto con un paziente perfettamente sano

In [ ]:
predictions = model.predict(x_test)

In [ ]:

b="si"
while b=="si":
    a=input("quale immagine vuoi che legga? fornire un numero da 0 a 10.000:")
    
    print("io ho interpretato l'immagine numero {} come: {}".format(a,np.argmax(predictions[int(a)])))
    print("\necco la predizione che mi ha permesso di fare questa scelta:\n\n",predictions[int(a)])
    #print(np.sum(predictions[int(a)]))
    print("\nla risposta corretta è:",y_test1[int(a)])
    print("\n il paziente modello avrebbe risposto: {}\n".format(np.argmax(cnn_predictions[int(a)])))
    print("\nvuoi che mostri l'immagine selezionata?\n") 
    if input()=='si':
        plt.imshow(x_test1[int(a)], cmap=plt.cm.binary)
        plt.show()
    
    for i in range(9999):
        predictions[int(i)]=np.argmax(predictions[int(i)])
        cnn_predictions[int(i)]=np.argmax(cnn_predictions[int(i)])
        
    #print( predictions)
    uguali =np.sum(np.array(cnn_predictions) == np.array(predictions))
    percentuale = (uguali / cnn_predictions.size) * 100

    print("I due pazienti hanno fornito un numero di risposte uguali pari a: {},\n il 'paziente in esame' ha fornito il {:.2f}% delle risposte che il paziente modello ha fornito.".format(uguali, percentuale))

    b=input("\nvuoi che legga un'altra immagine?\n")
    
print("se non è 'si' per me è no ;), alla prossima!")
